# Cross-Artifact Consistency Analysis
## Specification Quality Analysis for ActualGameSearch V3

This notebook performs a comprehensive cross-artifact consistency and quality analysis across `spec.md`, `plan.md`, and `tasks.md` following the analyze.prompt.md instructions.

**Analysis Scope:**
- Feature Directory: `specs/002-we-intend-to/`
- Constitution Authority: `.specify/memory/constitution.md`
- Detection Categories: Duplication, Ambiguity, Underspecification, Constitution Alignment, Coverage Gaps, Inconsistencies

**Analysis Goal:** Identify issues before implementation and ensure docs/contracts align with current code reality.

In [1]:
# Section 1: Execute Prerequisites Check
import os
import subprocess
import json
import sys
from pathlib import Path

# Change to repo root for prerequisites check
repo_root = "/workspaces/ActualGameSearch_V3"
os.chdir(repo_root)

print(f"Working directory: {os.getcwd()}")
print(f"Repository root: {repo_root}")

# Execute prerequisites check to get FEATURE_DIR and AVAILABLE_DOCS
try:
    # Make script executable first
    script_path = "./.specify/scripts/bash/check-prerequisites.sh"
    subprocess.run(["chmod", "+x", script_path], check=True)
    
    # Run the prerequisites check
    result = subprocess.run([
        script_path, 
        "--json", 
        "--require-tasks", 
        "--include-tasks"
    ], capture_output=True, text=True, check=True)
    
    prerequisites_data = json.loads(result.stdout)
    FEATURE_DIR = prerequisites_data["FEATURE_DIR"]
    AVAILABLE_DOCS = prerequisites_data["AVAILABLE_DOCS"]
    
    print("Prerequisites check PASSED")
    print(f"Feature Directory: {FEATURE_DIR}")
    print(f"Available Documents: {AVAILABLE_DOCS}")
    
    # Derive absolute paths for analysis
    SPEC_PATH = os.path.join(FEATURE_DIR, "spec.md")
    PLAN_PATH = os.path.join(FEATURE_DIR, "plan.md") 
    TASKS_PATH = os.path.join(FEATURE_DIR, "tasks.md")
    CONSTITUTION_PATH = os.path.join(repo_root, ".specify/memory/constitution.md")
    
    # Verify all required files exist
    required_files = [SPEC_PATH, PLAN_PATH, TASKS_PATH, CONSTITUTION_PATH]
    missing_files = [f for f in required_files if not os.path.exists(f)]
    
    if missing_files:
        print(f"ERROR: Missing required files: {missing_files}")
        sys.exit(1)
    else:
        print("All required files found")
        
except subprocess.CalledProcessError as e:
    print(f"Prerequisites check FAILED: {e}")
    print(f"stdout: {e.stdout}")
    print(f"stderr: {e.stderr}")
    sys.exit(1)
except json.JSONDecodeError as e:
    print(f"Failed to parse prerequisites JSON: {e}")
    sys.exit(1)

Working directory: /workspaces/ActualGameSearch_V3
Repository root: /workspaces/ActualGameSearch_V3
Prerequisites check PASSED
Feature Directory: /workspaces/ActualGameSearch_V3/specs/002-we-intend-to
Available Documents: ['research.md', 'data-model.md', 'contracts/', 'quickstart.md', 'tasks.md']
All required files found


In [2]:
# Section 2: Load and Parse Artifacts
import re
from typing import Dict, List, Any, Optional

def load_file_content(path: str) -> str:
    """Load file content with error handling."""
    try:
        with open(path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        print(f"Error loading {path}: {e}")
        return ""

def parse_markdown_sections(content: str) -> Dict[str, str]:
    """Parse markdown content into sections based on headers."""
    sections = {}
    current_section = None
    current_content = []
    
    lines = content.split('\n')
    for line in lines:
        # Check for markdown headers
        if line.strip().startswith('#'):
            # Save previous section if exists
            if current_section:
                sections[current_section] = '\n'.join(current_content).strip()
            
            # Start new section
            current_section = line.strip().lstrip('#').strip()
            current_content = []
        else:
            if current_section:
                current_content.append(line)
    
    # Don't forget the last section
    if current_section:
        sections[current_section] = '\n'.join(current_content).strip()
    
    return sections

# Load all artifact files
print("Loading artifact files...")

spec_content = load_file_content(SPEC_PATH)
plan_content = load_file_content(PLAN_PATH)
tasks_content = load_file_content(TASKS_PATH)
constitution_content = load_file_content(CONSTITUTION_PATH)

print(f"Loaded spec.md: {len(spec_content)} characters")
print(f"Loaded plan.md: {len(plan_content)} characters") 
print(f"Loaded tasks.md: {len(tasks_content)} characters")
print(f"Loaded constitution.md: {len(constitution_content)} characters")

# Parse into sections
spec_sections = parse_markdown_sections(spec_content)
plan_sections = parse_markdown_sections(plan_content)
tasks_sections = parse_markdown_sections(tasks_content)
constitution_sections = parse_markdown_sections(constitution_content)

print(f"\nSpec sections: {list(spec_sections.keys())}")
print(f"Plan sections: {list(plan_sections.keys())}")
print(f"Tasks sections: {list(tasks_sections.keys())}")
print(f"Constitution sections: {list(constitution_sections.keys())}")

Loading artifact files...
Loaded spec.md: 11265 characters
Loaded plan.md: 7872 characters
Loaded tasks.md: 10821 characters
Loaded constitution.md: 12833 characters

Spec sections: ['Feature Specification: Actual Game Search (Hybrid Full‑Text + Semantic)', 'Execution Flow (main)', '⚡ Quick Guidelines', 'Section Requirements', 'For AI Generation', 'User Scenarios & Testing (mandatory)', 'Primary User Story', 'Acceptance Scenarios', 'Edge Cases', 'Requirements (mandatory)', 'Functional Requirements', 'Filtering & Constraints', 'Payload for Client Re‑Ranking (WHAT the client needs)', 'Key Entities (payload shapes, not implementation)', 'Review & Acceptance Checklist', 'Content Quality', 'Requirement Completeness', 'Execution Status', 'Reality Check (as of 2025-09-23)']
Plan sections: ['Implementation Plan: Actual Game Search (Hybrid Full‑Text + Semantic)', 'Execution Flow (/plan command scope)', 'Summary', 'Technical Context', 'Constitution Check', 'Project Structure', 'Documentation (th

In [3]:
# Section 3: Build Semantic Models
from dataclasses import dataclass
from typing import Set

@dataclass
class Requirement:
    key: str
    text: str
    requirement_type: str  # 'functional' or 'non-functional'
    location: str
    testable: bool = False
    
@dataclass 
class Task:
    id: str
    description: str
    phase: str
    parallel: bool
    file_paths: List[str]
    dependencies: List[str]
    location: str

@dataclass
class Finding:
    id: str
    category: str
    severity: str
    locations: List[str] 
    summary: str
    recommendation: str

def extract_requirements_from_spec(spec_sections: Dict[str, str]) -> List[Requirement]:
    """Extract functional and non-functional requirements from spec."""
    requirements = []
    
    # Look for requirements section
    req_section = spec_sections.get('Requirements (mandatory)', '')
    if not req_section:
        req_section = spec_sections.get('Requirements', '')
    
    functional_section = spec_sections.get('Functional Requirements', '')
    if functional_section or 'Functional Requirements' in req_section:
        # Parse FR-XXX patterns
        fr_matches = re.findall(r'(FR-\d+):\s*(.*?)(?=FR-\d+|$)', functional_section or req_section, re.DOTALL)
        for fr_id, text in fr_matches:
            key = fr_id.lower().replace('-', '_')
            requirements.append(Requirement(
                key=key, 
                text=text.strip(),
                requirement_type='functional',
                location=f'spec.md#functional-requirements',
                testable='MUST' in text or 'SHALL' in text
            ))
    
    # Parse filtering & constraints section  
    filter_section = spec_sections.get('Filtering & Constraints', '')
    if filter_section:
        fr_matches = re.findall(r'(FR-\d+):\s*(.*?)(?=FR-\d+|$)', filter_section, re.DOTALL)
        for fr_id, text in fr_matches:
            key = fr_id.lower().replace('-', '_')
            requirements.append(Requirement(
                key=key,
                text=text.strip(), 
                requirement_type='functional',
                location=f'spec.md#filtering-constraints',
                testable='MUST' in text or 'SHALL' in text
            ))
    
    return requirements

def extract_tasks_from_tasks_md(tasks_sections: Dict[str, str]) -> List[Task]:
    """Extract tasks from tasks.md file."""
    tasks = []
    
    # Look through all sections for task patterns
    for section_name, content in tasks_sections.items():
        if 'Phase' in section_name:
            # Extract T### patterns
            task_matches = re.findall(r'- \[.\] (T\d+).*?\n\s*(.+?)(?=\n\s*-|\n\n|\Z)', content, re.DOTALL)
            for task_id, description in task_matches:
                # Extract file paths mentioned
                file_paths = re.findall(r'`([^`]*\.cs|[^`]*\.md|[^`]*\.yaml|[^`]*\.json)`', description)
                file_paths.extend(re.findall(r'Files?: `([^`]+)`', description))
                file_paths.extend(re.findall(r'File: `([^`]+)`', description))
                
                # Check if parallel
                is_parallel = '[P]' in description
                
                # Extract dependencies  
                dependencies = []
                if 'Dependencies:' in description:
                    dep_match = re.search(r'Dependencies: (.+)', description)
                    if dep_match:
                        deps_text = dep_match.group(1)
                        dependencies = re.findall(r'T\d+', deps_text)
                
                tasks.append(Task(
                    id=task_id,
                    description=description.strip(),
                    phase=section_name,
                    parallel=is_parallel,
                    file_paths=file_paths,
                    dependencies=dependencies,
                    location=f'tasks.md#{section_name.lower().replace(" ", "-")}'
                ))
    
    return tasks

def extract_constitution_principles(constitution_sections: Dict[str, str]) -> Dict[str, str]:
    """Extract constitution principles and MUST statements."""
    principles = {}
    
    # Look for Core Principles section
    core_principles = constitution_sections.get('Core Principles', '')
    if core_principles:
        # Extract principle sections (### I., ### II., etc.)
        principle_matches = re.findall(r'### ([IV]+\..*?)\n(.*?)(?=###|\Z)', core_principles, re.DOTALL)
        for title, content in principle_matches:
            principles[title] = content.strip()
    
    return principles

# Build semantic models
print("Building semantic models...")

requirements = extract_requirements_from_spec(spec_sections)
tasks = extract_tasks_from_tasks_md(tasks_sections)
constitution_principles = extract_constitution_principles(constitution_sections)

print(f"Extracted {len(requirements)} requirements")
print(f"Extracted {len(tasks)} tasks")  
print(f"Extracted {len(constitution_principles)} constitution principles")

# Display sample extractions
print(f"\nSample requirements:")
for req in requirements[:3]:
    print(f"  {req.key}: {req.text[:100]}...")

print(f"\nSample tasks:")  
for task in tasks[:3]:
    print(f"  {task.id}: {task.description[:100]}...")

print(f"\nConstitution principles:")
for title, content in list(constitution_principles.items())[:2]:
    print(f"  {title}: {content[:100]}...")

Building semantic models...
Extracted 22 requirements
Extracted 28 tasks
Extracted 0 constitution principles

Sample requirements:
  fr_001: The system MUST accept a free-text search prompt and return a candidate set suitable for client-side...
  fr_002: The client MUST present an initial subset of candidates with title, image (if available), short desc...
  fr_003: The system MUST support a “similar games” action from any result, returning a related set with brief...

Sample tasks:
  T001: - `ActualGameSearch.sln` with projects: `ActualGameSearch.AppHost`, `ActualGameSearch.Api`, `ActualG...
  T002: - Add Cosmos DB emulator resource; add database/containers (games, reviews)...
  T003: - `src/ActualGameSearch.ServiceDefaults/` with `AddServiceDefaults()`; enable OpenTelemetry, service...

Constitution principles:


In [4]:
# Section 4: Run Detection Passes
import difflib

def detect_duplications(requirements: List[Requirement]) -> List[Finding]:
    """Detect near-duplicate requirements."""
    findings = []
    
    for i, req1 in enumerate(requirements):
        for j, req2 in enumerate(requirements[i+1:], i+1):
            # Calculate similarity
            similarity = difflib.SequenceMatcher(None, req1.text.lower(), req2.text.lower()).ratio()
            
            if similarity > 0.7:  # High similarity threshold
                findings.append(Finding(
                    id=f"D{len(findings)+1:02d}",
                    category="Duplication", 
                    severity="HIGH",
                    locations=[req1.location, req2.location],
                    summary=f"Requirements {req1.key} and {req2.key} are {similarity:.1%} similar",
                    recommendation=f"Merge similar requirements; keep clearer version"
                ))
    
    return findings

def detect_ambiguities(requirements: List[Requirement], tasks: List[Task]) -> List[Finding]:
    """Detect vague language and unresolved placeholders.""" 
    findings = []
    
    # Vague adjectives to flag
    vague_terms = ['fast', 'scalable', 'secure', 'intuitive', 'robust', 'efficient', 'good', 'better']
    
    # Check requirements for vague terms
    for req in requirements:
        for term in vague_terms:
            if term in req.text.lower() and ('measurable' not in req.text.lower() and 'specific' not in req.text.lower()):
                findings.append(Finding(
                    id=f"A{len(findings)+1:02d}",
                    category="Ambiguity",
                    severity="MEDIUM", 
                    locations=[req.location],
                    summary=f"Requirement {req.key} uses vague term '{term}' without measurable criteria",
                    recommendation=f"Replace '{term}' with specific, measurable criteria"
                ))
    
    # Check for placeholder patterns
    placeholder_patterns = ['TODO', 'TKTK', '???', '<placeholder>', 'NEEDS CLARIFICATION']
    all_content = spec_content + plan_content + tasks_content
    
    for pattern in placeholder_patterns:
        if pattern in all_content:
            findings.append(Finding(
                id=f"A{len(findings)+1:02d}",
                category="Ambiguity",
                severity="HIGH",
                locations=["multiple files"],
                summary=f"Unresolved placeholder '{pattern}' found in artifacts",
                recommendation=f"Resolve all '{pattern}' markers before implementation"
            ))
    
    return findings

def detect_underspecification(requirements: List[Requirement], tasks: List[Task]) -> List[Finding]:
    """Detect underspecified requirements and tasks."""
    findings = []
    
    # Check for untestable requirements
    for req in requirements:
        if not req.testable and 'SHOULD' not in req.text and 'MAY' not in req.text:
            # Look for action verbs without measurable outcomes
            has_verb = any(verb in req.text.lower() for verb in ['must', 'shall', 'will', 'should', 'can'])
            has_outcome = any(outcome in req.text.lower() for outcome in ['return', 'display', 'show', 'log', 'store', 'validate'])
            
            if has_verb and not has_outcome:
                findings.append(Finding(
                    id=f"U{len(findings)+1:02d}",
                    category="Underspecification",
                    severity="MEDIUM",
                    locations=[req.location],
                    summary=f"Requirement {req.key} has action verb but missing measurable outcome",
                    recommendation="Add specific, testable acceptance criteria"
                ))
    
    # Check for tasks referencing undefined components
    for task in tasks:
        # Look for references to files/components not defined elsewhere
        undefined_refs = []
        for file_path in task.file_paths:
            if not any(file_path in spec_content or file_path in plan_content for file_path in task.file_paths):
                undefined_refs.append(file_path)
        
        if undefined_refs:
            findings.append(Finding(
                id=f"U{len(findings)+1:02d}",
                category="Underspecification", 
                severity="MEDIUM",
                locations=[task.location],
                summary=f"Task {task.id} references undefined files: {undefined_refs}",
                recommendation="Define referenced components in spec or plan"
            ))
    
    return findings

def detect_constitution_violations(requirements: List[Requirement], constitution_principles: Dict[str, str]) -> List[Finding]:
    """Detect violations of constitutional principles."""
    findings = []
    
    # Extract MUST statements from constitution
    must_statements = []
    for title, content in constitution_principles.items():
        must_matches = re.findall(r'([^.]*MUST[^.]*\.)', content)
        must_statements.extend([(title, stmt.strip()) for stmt in must_matches])
    
    # Check each requirement against MUST statements
    for req in requirements:
        # Check specific violations
        if 'test' in req.text.lower() and 'MUST' in req.text:
            # Check if violates "Test‑First" principle
            test_principle = "Test‑First, Observability, and Simplicity"
            if test_principle in constitution_principles:
                if 'before' not in req.text.lower() and 'first' not in req.text.lower():
                    findings.append(Finding(
                        id=f"C{len(findings)+1:02d}",
                        category="Constitution",
                        severity="CRITICAL",
                        locations=[req.location, "constitution.md#test-first"],
                        summary=f"Requirement {req.key} may violate Test-First principle",
                        recommendation="Ensure test requirements specify test-first approach"
                    ))
    
    return findings

def detect_coverage_gaps(requirements: List[Requirement], tasks: List[Task]) -> List[Finding]:
    """Detect requirements without task coverage and tasks without requirement mapping."""
    findings = []
    
    # Build requirement-to-task mapping
    req_task_map = {req.key: [] for req in requirements}
    unmapped_tasks = []
    
    for task in tasks:
        mapped = False
        for req in requirements:
            # Simple keyword matching for coverage
            req_keywords = set(re.findall(r'\w+', req.text.lower()))
            task_keywords = set(re.findall(r'\w+', task.description.lower()))
            
            # Check for overlap or explicit FR references
            if (len(req_keywords.intersection(task_keywords)) >= 2 or 
                req.key.upper() in task.description or
                req.key.replace('_', '-').upper() in task.description):
                req_task_map[req.key].append(task.id)
                mapped = True
        
        if not mapped:
            unmapped_tasks.append(task)
    
    # Find requirements with zero coverage
    for req in requirements:
        if not req_task_map[req.key] and req.requirement_type == 'functional':
            findings.append(Finding(
                id=f"G{len(findings)+1:02d}",
                category="Coverage Gap",
                severity="HIGH" if 'MUST' in req.text else "MEDIUM",
                locations=[req.location],
                summary=f"Requirement {req.key} has no associated tasks",
                recommendation=f"Add implementation tasks for requirement {req.key}"
            ))
    
    # Report unmapped tasks
    for task in unmapped_tasks[:5]:  # Limit to first 5 to avoid noise
        findings.append(Finding(
            id=f"G{len(findings)+1:02d}",
            category="Coverage Gap",
            severity="LOW",
            locations=[task.location],
            summary=f"Task {task.id} not mapped to any requirement",
            recommendation=f"Link task {task.id} to specific requirement or mark as infrastructure"
        ))
    
    return findings, req_task_map

def detect_inconsistencies(spec_sections: Dict[str, str], plan_sections: Dict[str, str], tasks: List[Task]) -> List[Finding]:
    """Detect terminology drift and inconsistencies across artifacts.""" 
    findings = []
    
    # Check for terminology variations
    terminology_groups = [
        ['game', 'title', 'app'],  # Game references
        ['review', 'comment', 'feedback'],  # Review references  
        ['search', 'query', 'lookup'],  # Search references
        ['candidate', 'result', 'item'],  # Result references
    ]
    
    all_text = spec_content + plan_content + tasks_content
    
    for group in terminology_groups:
        used_terms = [term for term in group if term in all_text.lower()]
        if len(used_terms) > 2:  # More than 2 variants used
            findings.append(Finding(
                id=f"I{len(findings)+1:02d}",
                category="Inconsistency",
                severity="LOW",
                locations=["multiple files"],
                summary=f"Multiple terms used for same concept: {used_terms}",
                recommendation=f"Standardize on single term from: {used_terms}"
            ))
    
    return findings

# Run all detection passes
print("Running detection passes...")

duplication_findings = detect_duplications(requirements)
ambiguity_findings = detect_ambiguities(requirements, tasks)
underspec_findings = detect_underspecification(requirements, tasks)
constitution_findings = detect_constitution_violations(requirements, constitution_principles)
coverage_findings, req_task_map = detect_coverage_gaps(requirements, tasks) 
inconsistency_findings = detect_inconsistencies(spec_sections, plan_sections, tasks)

all_findings = (duplication_findings + ambiguity_findings + underspec_findings + 
               constitution_findings + coverage_findings + inconsistency_findings)

print(f"Detection complete:")
print(f"  Duplication issues: {len(duplication_findings)}")
print(f"  Ambiguity issues: {len(ambiguity_findings)}")
print(f"  Underspecification issues: {len(underspec_findings)}")
print(f"  Constitution violations: {len(constitution_findings)}")
print(f"  Coverage gaps: {len(coverage_findings)}")
print(f"  Inconsistencies: {len(inconsistency_findings)}")
print(f"  Total findings: {len(all_findings)}")

Running detection passes...
Detection complete:
  Duplication issues: 0
  Ambiguity issues: 1
  Underspecification issues: 4
  Constitution violations: 0
  Coverage gaps: 12
  Inconsistencies: 2
  Total findings: 19


In [5]:
# Section 5: Assign Severity Levels
def categorize_by_severity(findings: List[Finding]) -> Dict[str, List[Finding]]:
    """Categorize findings by severity level."""
    severity_groups = {
        'CRITICAL': [],
        'HIGH': [],
        'MEDIUM': [],
        'LOW': []
    }
    
    for finding in findings:
        severity_groups[finding.severity].append(finding)
    
    return severity_groups

def validate_severity_assignment(findings: List[Finding], constitution_principles: Dict[str, str]) -> None:
    """Validate and adjust severity based on constitution impact."""
    
    for finding in findings:
        # Constitution violations are always CRITICAL
        if finding.category == "Constitution":
            finding.severity = "CRITICAL"
        
        # Missing core functionality is HIGH
        elif finding.category == "Coverage Gap" and "MUST" in finding.summary:
            if finding.severity not in ["CRITICAL", "HIGH"]:
                finding.severity = "HIGH"
        
        # Ambiguities in security/performance are HIGH  
        elif finding.category == "Ambiguity" and any(term in finding.summary.lower() 
                                                   for term in ['security', 'performance', 'latency']):
            if finding.severity == "MEDIUM":
                finding.severity = "HIGH"

# Apply severity validation
validate_severity_assignment(all_findings, constitution_principles)

# Categorize by severity
severity_groups = categorize_by_severity(all_findings)

print("Severity Distribution:")
for severity, findings in severity_groups.items():
    print(f"  {severity}: {len(findings)} issues")
    if findings:
        print(f"    Categories: {set(f.category for f in findings)}")

# Show critical issues first
if severity_groups['CRITICAL']:
    print(f"\n🚨 CRITICAL Issues (MUST be resolved before /implement):")
    for finding in severity_groups['CRITICAL']:
        print(f"  {finding.id}: {finding.summary}")

if severity_groups['HIGH']:
    print(f"\n⚠️  HIGH Priority Issues:")
    for finding in severity_groups['HIGH'][:5]:  # Show first 5
        print(f"  {finding.id}: {finding.summary}")
        
print(f"\nTotal findings by category:")
category_counts = {}
for finding in all_findings:
    category_counts[finding.category] = category_counts.get(finding.category, 0) + 1

for category, count in sorted(category_counts.items()):
    print(f"  {category}: {count}")

Severity Distribution:
  CRITICAL: 0 issues
  HIGH: 6 issues
    Categories: {'Coverage Gap', 'Ambiguity'}
  MEDIUM: 6 issues
    Categories: {'Underspecification', 'Coverage Gap'}
  LOW: 7 issues
    Categories: {'Coverage Gap', 'Inconsistency'}

⚠️  HIGH Priority Issues:
  A01: Unresolved placeholder 'NEEDS CLARIFICATION' found in artifacts
  G01: Requirement fr_003 has no associated tasks
  G02: Requirement fr_005 has no associated tasks
  G03: Requirement fr_007 has no associated tasks
  G06: Requirement fr_011 has no associated tasks

Total findings by category:
  Ambiguity: 1
  Coverage Gap: 12
  Inconsistency: 2
  Underspecification: 4


In [6]:
# Section 6: Generate Analysis Report
from IPython.display import Markdown, display

def generate_findings_table(findings: List[Finding], limit: int = 50) -> str:
    """Generate markdown table of findings."""
    
    # Limit to top findings (by severity order: CRITICAL, HIGH, MEDIUM, LOW)
    severity_order = {'CRITICAL': 0, 'HIGH': 1, 'MEDIUM': 2, 'LOW': 3}
    sorted_findings = sorted(findings, key=lambda f: (severity_order[f.severity], f.id))[:limit]
    
    table_lines = [
        "| ID | Category | Severity | Location(s) | Summary | Recommendation |",
        "|----|----------|----------|-------------|---------|----------------|"
    ]
    
    for finding in sorted_findings:
        locations_str = ", ".join(finding.locations) if len(finding.locations) <= 2 else f"{finding.locations[0]} +{len(finding.locations)-1} more"
        summary_short = finding.summary[:80] + "..." if len(finding.summary) > 80 else finding.summary
        recommendation_short = finding.recommendation[:60] + "..." if len(finding.recommendation) > 60 else finding.recommendation
        
        table_lines.append(f"| {finding.id} | {finding.category} | {finding.severity} | {locations_str} | {summary_short} | {recommendation_short} |")
    
    return "\n".join(table_lines)

def generate_coverage_summary(requirements: List[Requirement], req_task_map: Dict[str, List[str]]) -> str:
    """Generate requirements coverage summary table."""
    
    table_lines = [
        "| Requirement Key | Has Task? | Task IDs | Notes |",
        "|-----------------|-----------|----------|-------|"
    ]
    
    for req in requirements:
        has_tasks = bool(req_task_map[req.key])
        task_ids = ", ".join(req_task_map[req.key]) if req_task_map[req.key] else "None"
        notes = "✅ Covered" if has_tasks else "❌ No coverage"
        
        table_lines.append(f"| {req.key} | {'Yes' if has_tasks else 'No'} | {task_ids} | {notes} |")
    
    return "\n".join(table_lines)

# Generate the main analysis report
report_sections = []

# Header
report_sections.append("# Specification Analysis Report")
report_sections.append(f"**Analysis Date:** {os.popen('date').read().strip()}")
report_sections.append(f"**Feature Directory:** `{FEATURE_DIR}`")
report_sections.append("")

# Executive Summary
critical_count = len(severity_groups['CRITICAL'])
high_count = len(severity_groups['HIGH']) 
total_count = len(all_findings)

if critical_count > 0:
    report_sections.append("## 🚨 Executive Summary")
    report_sections.append(f"**CRITICAL ISSUES FOUND:** {critical_count} issues must be resolved before implementation.")
    report_sections.append(f"Total issues: {total_count} (Critical: {critical_count}, High: {high_count})")
    report_sections.append("")
elif high_count > 5:
    report_sections.append("## ⚠️ Executive Summary") 
    report_sections.append(f"**HIGH PRIORITY ISSUES:** {high_count} issues should be addressed.")
    report_sections.append(f"Total issues: {total_count} (High: {high_count})")
    report_sections.append("")
else:
    report_sections.append("## ✅ Executive Summary")
    report_sections.append(f"**ANALYSIS PASSED:** No critical issues found. {total_count} total issues identified.")
    report_sections.append("")

# Main findings table
report_sections.append("## Findings Table")
findings_table = generate_findings_table(all_findings)
report_sections.append(findings_table)
report_sections.append("")

if len(all_findings) > 50:
    report_sections.append(f"*Note: Table limited to top 50 findings. {len(all_findings) - 50} additional findings exist.*")
    report_sections.append("")

# Coverage summary
report_sections.append("## Coverage Summary")
coverage_table = generate_coverage_summary(requirements, req_task_map)
report_sections.append(coverage_table)
report_sections.append("")

# Constitution alignment
if constitution_findings:
    report_sections.append("## Constitution Alignment Issues")
    for finding in constitution_findings:
        report_sections.append(f"- **{finding.id}:** {finding.summary}")
        report_sections.append(f"  - *Recommendation:* {finding.recommendation}")
    report_sections.append("")

# Generate final report markdown
report_markdown = "\n".join(report_sections)

# Display the report
display(Markdown(report_markdown))

print("Report generated successfully!")

# Specification Analysis Report
**Analysis Date:** Wed Sep 24 14:35:30 UTC 2025
**Feature Directory:** `/workspaces/ActualGameSearch_V3/specs/002-we-intend-to`

## ⚠️ Executive Summary
**HIGH PRIORITY ISSUES:** 6 issues should be addressed.
Total issues: 19 (High: 6)

## Findings Table
| ID | Category | Severity | Location(s) | Summary | Recommendation |
|----|----------|----------|-------------|---------|----------------|
| A01 | Ambiguity | HIGH | multiple files | Unresolved placeholder 'NEEDS CLARIFICATION' found in artifacts | Resolve all 'NEEDS CLARIFICATION' markers before implementat... |
| G01 | Coverage Gap | HIGH | spec.md#functional-requirements | Requirement fr_003 has no associated tasks | Add implementation tasks for requirement fr_003 |
| G02 | Coverage Gap | HIGH | spec.md#functional-requirements | Requirement fr_005 has no associated tasks | Add implementation tasks for requirement fr_005 |
| G03 | Coverage Gap | HIGH | spec.md#functional-requirements | Requirement fr_007 has no associated tasks | Add implementation tasks for requirement fr_007 |
| G06 | Coverage Gap | HIGH | spec.md#functional-requirements | Requirement fr_011 has no associated tasks | Add implementation tasks for requirement fr_011 |
| G07 | Coverage Gap | HIGH | spec.md#filtering-constraints | Requirement fr_028 has no associated tasks | Add implementation tasks for requirement fr_028 |
| G04 | Coverage Gap | MEDIUM | spec.md#functional-requirements | Requirement fr_009 has no associated tasks | Add implementation tasks for requirement fr_009 |
| G05 | Coverage Gap | MEDIUM | spec.md#functional-requirements | Requirement fr_010 has no associated tasks | Add implementation tasks for requirement fr_010 |
| U01 | Underspecification | MEDIUM | tasks.md#phase-3.2:-tests-first-(tdd)-⚠️-must-complete-before-3.3 | Task T006 references undefined files: ['tests/ActualGameSearch.ContractTests/Gam... | Define referenced components in spec or plan |
| U02 | Underspecification | MEDIUM | tasks.md#phase-3.2:-tests-first-(tdd)-⚠️-must-complete-before-3.3 | Task T007 references undefined files: ['tests/ActualGameSearch.ContractTests/Rev... | Define referenced components in spec or plan |
| U03 | Underspecification | MEDIUM | tasks.md#phase-3.2:-tests-first-(tdd)-⚠️-must-complete-before-3.3 | Task T008 references undefined files: ['tests/ActualGameSearch.ContractTests/Sea... | Define referenced components in spec or plan |
| U04 | Underspecification | MEDIUM | tasks.md#phase-3.5:-polish | Task T027 references undefined files: ['docs/data-dictionary.md'] | Define referenced components in spec or plan |
| G08 | Coverage Gap | LOW | tasks.md#phase-3.1:-setup | Task T002 not mapped to any requirement | Link task T002 to specific requirement or mark as infrastruc... |
| G09 | Coverage Gap | LOW | tasks.md#phase-3.1:-setup | Task T003 not mapped to any requirement | Link task T003 to specific requirement or mark as infrastruc... |
| G10 | Coverage Gap | LOW | tasks.md#phase-3.1:-setup | Task T004 not mapped to any requirement | Link task T004 to specific requirement or mark as infrastruc... |
| G11 | Coverage Gap | LOW | tasks.md#phase-3.2:-tests-first-(tdd)-⚠️-must-complete-before-3.3 | Task T005 not mapped to any requirement | Link task T005 to specific requirement or mark as infrastruc... |
| G12 | Coverage Gap | LOW | tasks.md#phase-3.2:-tests-first-(tdd)-⚠️-must-complete-before-3.3 | Task T006 not mapped to any requirement | Link task T006 to specific requirement or mark as infrastruc... |
| I01 | Inconsistency | LOW | multiple files | Multiple terms used for same concept: ['game', 'title', 'app'] | Standardize on single term from: ['game', 'title', 'app'] |
| I02 | Inconsistency | LOW | multiple files | Multiple terms used for same concept: ['candidate', 'result', 'item'] | Standardize on single term from: ['candidate', 'result', 'it... |

## Coverage Summary
| Requirement Key | Has Task? | Task IDs | Notes |
|-----------------|-----------|----------|-------|
| fr_001 | Yes | T009, T014, T015, T017, T021, T023 | ✅ Covered |
| fr_002 | Yes | T009, T023 | ✅ Covered |
| fr_003 | No | None | ❌ No coverage |
| fr_004 | Yes | T009, T023 | ✅ Covered |
| fr_005 | No | None | ❌ No coverage |
| fr_006 | Yes | T009, T010, T014, T015, T021, T023 | ✅ Covered |
| fr_007 | No | None | ❌ No coverage |
| fr_008 | Yes | T014, T021 | ✅ Covered |
| fr_009 | No | None | ❌ No coverage |
| fr_010 | No | None | ❌ No coverage |
| fr_011 | No | None | ❌ No coverage |
| fr_012 | Yes | T009, T015, T017, T023 | ✅ Covered |
| fr_013 | Yes | T009, T010, T014, T015, T017, T021, T023, T027 | ✅ Covered |
| fr_020 | Yes | T009, T010, T014, T015, T023 | ✅ Covered |
| fr_021 | Yes | T016 | ✅ Covered |
| fr_022 | Yes | T009, T010, T014, T015, T021, T023 | ✅ Covered |
| fr_023 | Yes | T009, T014, T015, T021, T023, T025 | ✅ Covered |
| fr_024 | Yes | T009, T023 | ✅ Covered |
| fr_025 | Yes | T014, T015, T016, T021 | ✅ Covered |
| fr_026 | Yes | T001, T009, T010, T014, T015, T016, T017, T018, T019, T020, T021, T023 | ✅ Covered |
| fr_027 | Yes | T009, T010, T014, T015, T016, T023, T025 | ✅ Covered |
| fr_028 | No | None | ❌ No coverage |


Report generated successfully!


In [7]:
# Section 7: Calculate Coverage Metrics
def calculate_coverage_metrics(requirements: List[Requirement], tasks: List[Task], 
                              req_task_map: Dict[str, List[str]], all_findings: List[Finding]) -> Dict[str, Any]:
    """Calculate comprehensive coverage and quality metrics."""
    
    # Basic counts
    total_requirements = len(requirements)
    functional_requirements = len([r for r in requirements if r.requirement_type == 'functional'])
    non_functional_requirements = len([r for r in requirements if r.requirement_type == 'non-functional'])
    total_tasks = len(tasks)
    
    # Coverage metrics
    requirements_with_tasks = len([req for req in requirements if req_task_map[req.key]])
    coverage_percentage = (requirements_with_tasks / total_requirements * 100) if total_requirements > 0 else 0
    
    # Quality metrics
    ambiguity_count = len([f for f in all_findings if f.category == "Ambiguity"])
    duplication_count = len([f for f in all_findings if f.category == "Duplication"])
    critical_issues_count = len([f for f in all_findings if f.severity == "CRITICAL"])
    high_issues_count = len([f for f in all_findings if f.severity == "HIGH"])
    
    # Task distribution
    phase_distribution = {}
    parallel_tasks = 0
    for task in tasks:
        phase_distribution[task.phase] = phase_distribution.get(task.phase, 0) + 1
        if task.parallel:
            parallel_tasks += 1
    
    # Constitution compliance
    constitution_violations = len([f for f in all_findings if f.category == "Constitution"])
    
    return {
        'total_requirements': total_requirements,
        'functional_requirements': functional_requirements, 
        'non_functional_requirements': non_functional_requirements,
        'total_tasks': total_tasks,
        'requirements_with_tasks': requirements_with_tasks,
        'coverage_percentage': coverage_percentage,
        'ambiguity_count': ambiguity_count,
        'duplication_count': duplication_count,
        'critical_issues_count': critical_issues_count,
        'high_issues_count': high_issues_count,
        'total_findings': len(all_findings),
        'phase_distribution': phase_distribution,
        'parallel_tasks': parallel_tasks,
        'constitution_violations': constitution_violations
    }

# Calculate metrics
metrics = calculate_coverage_metrics(requirements, tasks, req_task_map, all_findings)

print("📊 Coverage and Quality Metrics:")
print(f"  Total Requirements: {metrics['total_requirements']} (Functional: {metrics['functional_requirements']}, Non-functional: {metrics['non_functional_requirements']})")
print(f"  Total Tasks: {metrics['total_tasks']} (Parallel: {metrics['parallel_tasks']})")
print(f"  Coverage: {metrics['coverage_percentage']:.1f}% ({metrics['requirements_with_tasks']}/{metrics['total_requirements']} requirements have tasks)")
print(f"  Quality Issues: {metrics['total_findings']} total ({metrics['critical_issues_count']} critical, {metrics['high_issues_count']} high)")
print(f"  Constitution Violations: {metrics['constitution_violations']}")
print(f"  Ambiguities: {metrics['ambiguity_count']}")
print(f"  Duplications: {metrics['duplication_count']}")

print(f"\n📋 Task Distribution by Phase:")
for phase, count in sorted(metrics['phase_distribution'].items()):
    print(f"  {phase}: {count} tasks")

# Create metrics summary for report
metrics_section = f"""
## Metrics Summary

**Requirements Coverage:**
- Total Requirements: {metrics['total_requirements']} (Functional: {metrics['functional_requirements']}, Non-functional: {metrics['non_functional_requirements']})
- Requirements with Tasks: {metrics['requirements_with_tasks']}/{metrics['total_requirements']}
- Coverage Percentage: {metrics['coverage_percentage']:.1f}%

**Task Analysis:**
- Total Tasks: {metrics['total_tasks']}
- Parallel Tasks: {metrics['parallel_tasks']}

**Quality Assessment:**
- Total Findings: {metrics['total_findings']}
- Critical Issues: {metrics['critical_issues_count']}
- High Priority Issues: {metrics['high_issues_count']}
- Constitution Violations: {metrics['constitution_violations']}
- Ambiguity Count: {metrics['ambiguity_count']}
- Duplication Count: {metrics['duplication_count']}

**Task Distribution:**
"""

for phase, count in sorted(metrics['phase_distribution'].items()):
    metrics_section += f"- {phase}: {count} tasks\n"

display(Markdown(metrics_section))

📊 Coverage and Quality Metrics:
  Total Requirements: 22 (Functional: 22, Non-functional: 0)
  Total Tasks: 28 (Parallel: 0)
  Coverage: 68.2% (15/22 requirements have tasks)
  Quality Issues: 19 total (0 critical, 6 high)
  Constitution Violations: 0
  Ambiguities: 1
  Duplications: 0

📋 Task Distribution by Phase:
  Phase 3.1: Setup: 4 tasks
  Phase 3.2: Tests First (TDD) ⚠️ MUST COMPLETE BEFORE 3.3: 6 tasks
  Phase 3.3: Core Implementation (ONLY after tests are failing): 10 tasks
  Phase 3.4: Integration: 4 tasks
  Phase 3.5: Polish: 4 tasks



## Metrics Summary

**Requirements Coverage:**
- Total Requirements: 22 (Functional: 22, Non-functional: 0)
- Requirements with Tasks: 15/22
- Coverage Percentage: 68.2%

**Task Analysis:**
- Total Tasks: 28
- Parallel Tasks: 0

**Quality Assessment:**
- Total Findings: 19
- Critical Issues: 0
- High Priority Issues: 6
- Constitution Violations: 0
- Ambiguity Count: 1
- Duplication Count: 0

**Task Distribution:**
- Phase 3.1: Setup: 4 tasks
- Phase 3.2: Tests First (TDD) ⚠️ MUST COMPLETE BEFORE 3.3: 6 tasks
- Phase 3.3: Core Implementation (ONLY after tests are failing): 10 tasks
- Phase 3.4: Integration: 4 tasks
- Phase 3.5: Polish: 4 tasks


In [8]:
# Section 8: Provide Next Actions and Remediation Recommendations

def generate_next_actions(metrics: Dict[str, Any], severity_groups: Dict[str, List[Finding]], 
                         requirements: List[Requirement], tasks: List[Task]) -> str:
    """Generate next actions based on analysis results."""
    
    actions = []
    
    # Critical issues block implementation
    if metrics['critical_issues_count'] > 0:
        actions.append("## 🚨 IMMEDIATE ACTIONS REQUIRED")
        actions.append(f"**CRITICAL ISSUES MUST BE RESOLVED before `/implement` command.**")
        actions.append("")
        
        for finding in severity_groups['CRITICAL']:
            actions.append(f"1. **{finding.id}**: {finding.summary}")
            actions.append(f"   - Action: {finding.recommendation}")
            actions.append("")
        
        actions.append("**Recommended Commands:**")
        if any('Constitution' in f.category for f in severity_groups['CRITICAL']):
            actions.append("- Review constitution alignment and update spec/plan accordingly")
        actions.append("- `Run /specify` to refine requirements")
        actions.append("- `Run /plan` to adjust architecture if needed")
        actions.append("")
    
    # High priority recommendations
    elif metrics['high_issues_count'] > 0:
        actions.append("## ⚠️ HIGH PRIORITY RECOMMENDATIONS")
        actions.append(f"**{metrics['high_issues_count']} high-priority issues should be addressed.**")
        actions.append("You may proceed to implementation, but consider resolving these first:")
        actions.append("")
        
        for finding in severity_groups['HIGH'][:5]:  # Show top 5
            actions.append(f"- **{finding.id}**: {finding.summary}")
            actions.append(f"  - Suggested: {finding.recommendation}")
            actions.append("")
    
    # Medium/Low issues guidance
    if metrics['coverage_percentage'] < 80:
        actions.append("## 📋 COVERAGE IMPROVEMENT")
        uncovered_reqs = [req for req in requirements if not req_task_map[req.key]]
        actions.append(f"**Coverage is {metrics['coverage_percentage']:.1f}%** - {len(uncovered_reqs)} requirements lack task coverage.")
        actions.append("")
        actions.append("Uncovered requirements:")
        for req in uncovered_reqs[:10]:  # Show first 10
            actions.append(f"- `{req.key}`: {req.text[:60]}...")
        actions.append("")
        actions.append("**Recommended:** Manually edit `tasks.md` to add coverage for missing requirements.")
        actions.append("")
    
    # Success case
    if metrics['critical_issues_count'] == 0 and metrics['high_issues_count'] <= 2:
        actions.append("## ✅ READY TO PROCEED")
        actions.append("**Analysis PASSED** - No critical blockers found.")
        actions.append("")
        actions.append("**Recommended Next Steps:**")
        actions.append("1. Run `/implement` to begin implementation")
        actions.append("2. Address remaining issues during implementation as needed")
        actions.append("")
    
    # Implementation readiness checklist
    actions.append("## 🔍 IMPLEMENTATION READINESS CHECKLIST")
    actions.append(f"- [{'x' if metrics['critical_issues_count'] == 0 else ' '}] No critical constitution violations")
    actions.append(f"- [{'x' if metrics['coverage_percentage'] >= 70 else ' '}] Requirements coverage ≥ 70% ({metrics['coverage_percentage']:.1f}%)")
    actions.append(f"- [{'x' if metrics['ambiguity_count'] <= 5 else ' '}] Ambiguities manageable (≤ 5, found {metrics['ambiguity_count']})")
    actions.append(f"- [{'x' if metrics['duplication_count'] == 0 else ' '}] No requirement duplications ({metrics['duplication_count']} found)")
    actions.append("")
    
    # Specific command recommendations
    actions.append("## 🛠️ SPECIFIC COMMAND RECOMMENDATIONS")
    
    if metrics['constitution_violations'] > 0:
        actions.append("- `Run /specify` with constitutional compliance focus")
        
    if metrics['coverage_percentage'] < 70:
        actions.append(f"- Manually edit `tasks.md` to add missing coverage")
        
    if metrics['ambiguity_count'] > 5:
        actions.append("- Run `/specify` to clarify ambiguous requirements")
        
    if metrics['duplication_count'] > 0:
        actions.append("- Run `/specify` to consolidate duplicate requirements")
    
    if len([f for f in all_findings if f.category == "Inconsistency"]) > 3:
        actions.append("- Run `/plan` to standardize terminology across artifacts")
    
    actions.append("")
    
    return "\n".join(actions)

# Generate next actions
next_actions = generate_next_actions(metrics, severity_groups, requirements, tasks)

# Display next actions
display(Markdown(next_actions))

# Final user prompt
print("\n" + "="*80)
print("📋 ANALYSIS COMPLETE")
print("="*80)

if metrics['critical_issues_count'] > 0:
    print(f"⚠️  BLOCKED: {metrics['critical_issues_count']} critical issues must be resolved before implementation.")
    print("   Recommendation: Address critical issues first, then re-run analysis.")
elif metrics['high_issues_count'] > 5:
    print(f"⚠️  PROCEED WITH CAUTION: {metrics['high_issues_count']} high-priority issues found.")
    print("   Recommendation: Consider addressing major issues before implementation.")
else:
    print("✅ READY TO PROCEED: No critical blockers found.")
    print("   Recommendation: You may proceed with /implement.")

print(f"\nTotal Issues: {metrics['total_findings']} | Coverage: {metrics['coverage_percentage']:.1f}%")
print("\n" + "="*80)

# Ask user about remediation
print("\n🔧 Would you like me to suggest concrete remediation edits for the top issues?")
print("   (Note: I will NOT apply edits automatically - I will only suggest specific changes)")

# Store analysis results for potential follow-up
analysis_results = {
    'metrics': metrics,
    'findings': all_findings,
    'severity_groups': severity_groups,
    'requirements': requirements,
    'tasks': tasks,
    'req_task_map': req_task_map
}

print(f"\n💾 Analysis results stored in 'analysis_results' variable for further processing.")

## ⚠️ HIGH PRIORITY RECOMMENDATIONS
**6 high-priority issues should be addressed.**
You may proceed to implementation, but consider resolving these first:

- **A01**: Unresolved placeholder 'NEEDS CLARIFICATION' found in artifacts
  - Suggested: Resolve all 'NEEDS CLARIFICATION' markers before implementation

- **G01**: Requirement fr_003 has no associated tasks
  - Suggested: Add implementation tasks for requirement fr_003

- **G02**: Requirement fr_005 has no associated tasks
  - Suggested: Add implementation tasks for requirement fr_005

- **G03**: Requirement fr_007 has no associated tasks
  - Suggested: Add implementation tasks for requirement fr_007

- **G06**: Requirement fr_011 has no associated tasks
  - Suggested: Add implementation tasks for requirement fr_011

## 📋 COVERAGE IMPROVEMENT
**Coverage is 68.2%** - 7 requirements lack task coverage.

Uncovered requirements:
- `fr_003`: The system MUST support a “similar games” action from any re...
- `fr_005`: The system MUST provide actionable, non-technical error mess...
- `fr_007`: The system MUST enforce basic rate limiting to protect from ...
- `fr_009`: The system SHOULD allow exploration of related games via mul...
- `fr_010`: The system MAY support visual exploration of embeddings (2D/...
- `fr_011`: The experience MUST be responsive and accessible.
-...
- `fr_028`: The system MUST allow including/excluding “singleton” games ...

**Recommended:** Manually edit `tasks.md` to add coverage for missing requirements.

## 🔍 IMPLEMENTATION READINESS CHECKLIST
- [x] No critical constitution violations
- [ ] Requirements coverage ≥ 70% (68.2%)
- [x] Ambiguities manageable (≤ 5, found 1)
- [x] No requirement duplications (0 found)

## 🛠️ SPECIFIC COMMAND RECOMMENDATIONS
- Manually edit `tasks.md` to add missing coverage



📋 ANALYSIS COMPLETE
⚠️  PROCEED WITH CAUTION: 6 high-priority issues found.
   Recommendation: Consider addressing major issues before implementation.

Total Issues: 19 | Coverage: 68.2%


🔧 Would you like me to suggest concrete remediation edits for the top issues?
   (Note: I will NOT apply edits automatically - I will only suggest specific changes)

💾 Analysis results stored in 'analysis_results' variable for further processing.
